In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math

# Load dataset
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Utility functions
def entropy(y):
    """Calculate the entropy of the labels."""
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

def information_gain(X_column, y, threshold):
    """Calculate information gain based on a threshold split."""
    left_indices = X_column <= threshold
    right_indices = X_column > threshold
    if len(set(left_indices)) == 1:
        return 0
    
    # Calculate weighted average entropy after split
    n = len(y)
    n_left, n_right = sum(left_indices), sum(right_indices)
    e_left, e_right = entropy(y[left_indices]), entropy(y[right_indices])
    weighted_avg_entropy = (n_left/n) * e_left + (n_right/n) * e_right
    return entropy(y) - weighted_avg_entropy

class DecisionTreeNode:
    def __init__(self, level=0):
        self.level = level
        self.left = None
        self.right = None
        self.split_feature = None
        self.threshold = None
        self.entropy = None
        self.is_leaf = False
        self.prediction = None

def build_tree(X, y, level=0):
    node = DecisionTreeNode(level)
    node.entropy = entropy(y)
    
    # Base case: If all examples are of the same class
    if len(set(y)) == 1:
        node.is_leaf = True
        node.prediction = y[0]
        print(f"Level {level}")
        print(f"Count of {Counter(y)}")
        print(f"Current Entropy is = {node.entropy}")
        print("Reached leaf Node")
        return node

    # Find the best feature and threshold to split
    best_gain = -1
    n_samples, n_features = X.shape
    for feature_idx in range(n_features):
        X_column = X[:, feature_idx]
        thresholds = np.unique(X_column)
        for threshold in thresholds:
            gain = information_gain(X_column, y, threshold)
            if gain > best_gain:
                best_gain = gain
                node.split_feature = feature_idx
                node.threshold = threshold

    # Print the node information
    print(f"Level {level}")
    print(f"Count of {Counter(y)}")
    print(f"Current Entropy is = {node.entropy}")
    print(f"Splitting on feature {node.split_feature} with gain ratio {best_gain}")

    # Split the data
    left_indices = X[:, node.split_feature] <= node.threshold
    right_indices = X[:, node.split_feature] > node.threshold

    node.left = build_tree(X[left_indices], y[left_indices], level + 1)
    node.right = build_tree(X[right_indices], y[right_indices], level + 1)
    
    return node

# Example run
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2, random_state=42)
root = build_tree(X_train, y_train)


Level 0
Count of Counter({np.int64(1): 41, np.int64(0): 40, np.int64(2): 39})
Current Entropy is = 1.5846619079379884
Splitting on feature 2 with gain ratio 0.9182958340544896
Level 1
Count of Counter({np.int64(0): 40})
Current Entropy is = -0.0
Reached leaf Node
Level 1
Count of Counter({np.int64(1): 41, np.int64(2): 39})
Current Entropy is = 0.9995491108252483
Splitting on feature 2 with gain ratio 0.6379119490346853
Level 2
Count of Counter({np.int64(1): 36, np.int64(2): 1})
Current Entropy is = 0.1792560669283215
Splitting on feature 3 with gain ratio 0.1792560669283215
Level 3
Count of Counter({np.int64(1): 36})
Current Entropy is = -0.0
Reached leaf Node
Level 3
Count of Counter({np.int64(2): 1})
Current Entropy is = -0.0
Reached leaf Node
Level 2
Count of Counter({np.int64(2): 38, np.int64(1): 5})
Current Entropy is = 0.5185697317883058
Splitting on feature 3 with gain ratio 0.1801704529380201
Level 3
Count of Counter({np.int64(2): 4, np.int64(1): 4})
Current Entropy is = 1.0
Sp

In [2]:
class DecisionTreeClassifier:
    def __init__(self):
        self.root = None
    
    def fit(self, X, y):
        self.root = build_tree(X, y)

    def predict(self, X):
        predictions = [self._traverse_tree(x, self.root) for x in X]
        return np.array(predictions)

    def _traverse_tree(self, x, node):
        if node.is_leaf:
            return node.prediction
        if x[node.split_feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

# Example usage:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy * 100}%")


Level 0
Count of Counter({np.int64(1): 41, np.int64(0): 40, np.int64(2): 39})
Current Entropy is = 1.5846619079379884
Splitting on feature 2 with gain ratio 0.9182958340544896
Level 1
Count of Counter({np.int64(0): 40})
Current Entropy is = -0.0
Reached leaf Node
Level 1
Count of Counter({np.int64(1): 41, np.int64(2): 39})
Current Entropy is = 0.9995491108252483
Splitting on feature 2 with gain ratio 0.6379119490346853
Level 2
Count of Counter({np.int64(1): 36, np.int64(2): 1})
Current Entropy is = 0.1792560669283215
Splitting on feature 3 with gain ratio 0.1792560669283215
Level 3
Count of Counter({np.int64(1): 36})
Current Entropy is = -0.0
Reached leaf Node
Level 3
Count of Counter({np.int64(2): 1})
Current Entropy is = -0.0
Reached leaf Node
Level 2
Count of Counter({np.int64(2): 38, np.int64(1): 5})
Current Entropy is = 0.5185697317883058
Splitting on feature 3 with gain ratio 0.1801704529380201
Level 3
Count of Counter({np.int64(2): 4, np.int64(1): 4})
Current Entropy is = 1.0
Sp